In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from math import ceil
from scanpy import read_h5ad
from tensorflow_probability import math as tm
tfk = tm.psd_kernels
import squidpy as sq
import scanpy as sc

In [ ]:
import sys
sys.path.append("/Users/user/nsf-paper/") # go to parent dir where nsf is installed

In [ ]:
from models import cf,sf,sfh
from models.mefisto import MEFISTO
from utils import preprocess,training,misc,visualize,postprocess

In [ ]:
random_state=123

In [ ]:
folder_path = "range_benchmark"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
else:
    print(f"Folder '{folder_path}' already exists.")

In [ ]:
histo_2 = pd.read_csv('../Histology_Visium_FFPE_Human_Prostate_Cancer_cloupe.csv').dropna()
histo_2.index = histo_2['Barcode']

In [ ]:
histo_2.index = histo_2['Barcode']
histo_2

In [ ]:
# read ST data
adata = sc.read_visium(path = '../invasive_prostate_visium/', 
                       count_file='Visium_FFPE_Human_Prostate_Cancer_filtered_feature_bc_matrix.h5', 
                       library_id='A1_spot',                        
                       load_images=True)
adata.var_names_make_unique()
adata.var['SYMBOL'] = adata.var_names
adata = adata[adata.obs_names.isin(histo_2['Barcode']),:]

In [ ]:
dtp = "float32"
pth ='.'
mpth = '.'

In [ ]:
import random
ad = adata

sc.pp.filter_genes(ad, min_cells=1)
sc.pp.filter_cells(ad, min_counts=100)
ad.layers = {"counts":ad.X.copy()} #store raw counts before normalization changes ad.X
sc.pp.normalize_total(ad, inplace=True, layers=None, key_added="sizefactor")
sc.pp.log1p(ad)

# %% normalization, feature selection and train/test split
ad.var['deviance_poisson'] = preprocess.deviancePoisson(ad.layers["counts"])
o = np.argsort(-ad.var['deviance_poisson'])
idx = list(range(ad.shape[0]))
random.shuffle(idx)
ad = ad[idx,o]

ad.write_h5ad(path.join(pth,"visium_prostate_cancer.h5ad"),compression="gzip")
ad2 = ad[:,:2000]
ad2.write_h5ad(path.join(pth,"visium_prostate_cancer_J2000.h5ad"),compression="gzip")

In [ ]:
J = 2000
ad = read_h5ad(path.join(pth,"visium_prostate_cancer_J{}.h5ad".format(J)))#[:,:J]
Dtr,Dval = preprocess.anndata_to_train_val(ad,train_frac=1,layer="counts",sz="scanpy")
Dtr_n,Dval_n = preprocess.anndata_to_train_val(ad,train_frac=1) #normalized data
fmeans,Dtr_c,Dval_c = preprocess.center_data(Dtr_n,Dval_n) #centered features
Xtr = Dtr["X"] #note this should be identical to Dtr_n["X"]
Ntr = Xtr.shape[0]
Dtf = preprocess.prepare_datasets_tf(Dtr,Dval=Dval,shuffle=False)
Dtf_n = preprocess.prepare_datasets_tf(Dtr_n,Dval=Dval_n,shuffle=False)
Dtf_c = preprocess.prepare_datasets_tf(Dtr_c,Dval=Dval_c,shuffle=False)
visualize.heatmap(Xtr,Dtr["Y"][:,0],marker="D",s=15)

In [ ]:
#%% Visualize raw data
import numpy as np
plt.imshow(np.log1p(Dtr["Y"])[:50,:100],cmap="Blues")

In [ ]:
#%% Visualize inducing points
Z = misc.kmeans_inducing_pts(Xtr,500)
fig,ax=plt.subplots(figsize=(12,10))
ax.scatter(Xtr[:,0],Xtr[:,1],marker="D",s=50,)
ax.scatter(Z[:,0],Z[:,1],c="red",s=30)

In [ ]:
# %% initialize inducing points and tuning parameters
Z = misc.kmeans_inducing_pts(Xtr, 3000)
M = Z.shape[0]
ker = tfk.MaternThreeHalves
S = 3 #samples for elbo approximation

In [ ]:
L = np.arange(8,51)
L

In [ ]:
# NSF: Spatial only with non-negative factors
for col in L:
    fit = sf.SpatialFactorization(J,col,Z,psd_kernel=ker,nonneg=True,lik="poi")
    fit.elbo_avg(Xtr,Dtr["Y"],sz=Dtr["sz"])
    fit.init_loadings(Dtr["Y"],X=Xtr,sz=Dtr["sz"])
    fit.elbo_avg(Xtr,Dtr["Y"],sz=Dtr["sz"])
    pp = fit.generate_pickle_path("scanpy",base=mpth)
    tro = training.ModelTrainer(fit,pickle_path=pp)


    hmkw = {"figsize":(4,4), "s":0.3, "marker":"D", "subplot_space":0,
        "spinecolor":"white"}
    insf = postprocess.interpret_nsf(fit,Xtr,S=10,lda_mode=False)
    tgnames = [str(i) for i in range(1,col+1)]

#%% Top genes for each latent dimensions
    W = insf["loadings"]#*insf["totals"][:,None]
    topgenes = W.argmax(axis=0).tolist()
    tgnames = ad.var.index[topgenes]
    Ytg = Dtr["Y"][:,topgenes]/Dtr["sz"]
    fig,axes=visualize.multiheatmap(Xtr, np.sqrt(Ytg), (4,3), **hmkw)
#save loadings to disk for further interpretation
    Wdf=pd.DataFrame(W*insf["totals"][:,None], index=ad.var.index, columns=range(1,col+1))
    W = insf["loadings"]#*insf["totals"][:,None]
    Wdf=pd.DataFrame(W*insf["totals"][:,None], index=ad.var.index, columns=range(1,col+1))
    pd.DataFrame(insf["factors"],index = ad.obs_names).to_csv(f'range_benchmark/factors_nsf_prostate_poi_{col}_bench.csv')

In [ ]:
#%% PNMF: Non-spatial, nonnegative
for col in L:
    fit = cf.CountFactorization(Ntr, J, col, lik="poi", nonneg=True)
    fit.elbo_avg(Dtr["Y"],sz=Dtr["sz"],idx=Dtr["idx"])
    fit.init_loadings(Dtr["Y"],sz=Dtr["sz"])
    pp = fit.generate_pickle_path("scanpy",base=mpth)
    tro = training.ModelTrainer(fit,pickle_path=pp)
    ttl = "PNMF: nonspatial, non-negative factors, Poisson likelihood"

    hmkw = {"figsize":(10,8), "s":0.5, "marker":"D", "subplot_space":0,
        "spinecolor":"white"}
    ipnmf = postprocess.interpret_pnmf(fit,S=8,lda_mode=False)
    tgnames = [str(i) for i in range(1,col+1)]
    
    pd.DataFrame(ipnmf["factors"],index = ad.obs_names).to_csv(f'range_benchmark/factors_pnmf_prostate_poi_{col}_bench.csv')


In [ ]:
#%% NSF Hybrid object
for col in L:
    fit = sfh.SpatialFactorizationHybrid(Ntr, J, col, Z, lik="poi", nonneg=True,
                                       psd_kernel=ker)
    fit.elbo_avg(Dtr["X"],Dtr["Y"],Dtr["idx"])
    fit.init_loadings(Dtr["Y"],X=Dtr["X"])
    pp = fit.generate_pickle_path("scanpy",base=mpth)
    tro = training.ModelTrainer(fit,pickle_path=pp)

    ttl = "NSFH: spatial, non-negative factors, Poisson likelihood"

    hmkw = {"figsize":(10,4), "s":0.5, "marker":"D", "subplot_space":0,
        "spinecolor":"white"}
    insfh = postprocess.interpret_nsfh(fit,Xtr,S=10,lda_mode=False)
    Ws = insfh['spatial']["loadings"]#*insf["totals"][:,None]
    Wdfs=pd.DataFrame(Ws*insfh["totals"][:,None], index=ad.var.index)
    
    Wns = insfh['nonspatial']["loadings"]#*insf["totals"][:,None]
    Wdfns=pd.DataFrame(Wns*insfh["totals"][:,None], index=ad.var.index)
    
    pd.DataFrame(insfh['nonspatial']["factors"],index = ad.obs_names).to_csv(f'range_benchmark/nonspatialfactors_nsfh_prostate_poi_{col}_bench.csv')
    pd.DataFrame(insfh['spatial']["factors"],index = ad.obs_names).to_csv(f'range_benchmark/spatialfactors_nsfh_prostate_poi_{col}_bench.csv')


In [ ]:
# %% MEFISTO- Gaussian
L =  np.arange(27,51)
for col in L:
    mef = MEFISTO(Dtr_n, col, inducing_pts=1000)
    ttl = "MEFISTO"
    dev_mef = visualize.gof(mef,Dtr,Dval=Dval,title=ttl)
    pd.DataFrame(mef.ent.model.nodes["Z"].getExpectations()["E"],index = ad.obs_names).to_csv(f'range_benchmark/new_factors_mefisto_prostate_{col}_bench.csv')

In [ ]:
adata = sc.read_visium(path = '../invasive_prostate_visium/', 
                       count_file='Visium_FFPE_Human_Prostate_Cancer_filtered_feature_bc_matrix.h5', 
                       library_id='A1_spot',                        
                       load_images=True)
adata.var_names_make_unique()
adata.var['SYMBOL'] = adata.var_names

In [ ]:
adata = adata[adata.obs_names.isin(histo_2['Barcode']),:]

In [ ]:
adata.obs['Histology']=histo_2['Histology']

In [ ]:
sc.pp.filter_genes(adata, min_cells=100)
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

In [ ]:
from sklearn import metrics
res = []

In [ ]:
import scanpy as sc

def find_leiden_resolution(
    adata,
    target_clusters=6,
    start_resolution=0.1,
    tolerance=0.01,
    max_iters=100,
    random_state=123,
    neighbors_key='cellpie',
    key_added='cellPie',
    use_rep = 'cellpie',
    
    n_neighbors=90
):
    resolution = start_resolution
    step = 0.01  # Step size for adjusting the resolution
    cluster_counts = []
    
    sc.pp.neighbors(adata, n_neighbors= n_neighbors, random_state=random_state, use_rep=use_rep, key_added=neighbors_key)
    
    for i in np.arange(start_resolution,1.01,step):
        
        sc.tl.leiden(adata, resolution=i, random_state=random_state, neighbors_key=neighbors_key, key_added=key_added)
        
        n_clusters = adata.obs[key_added].nunique()
        cluster_counts.append((resolution, n_clusters))
        
        if abs(n_clusters - target_clusters) <= tolerance:
            print(f"Found resolution: {i} with {n_clusters} clusters")
            return 
        else:
            continue

In [ ]:
L = np.arange(8,51)

In [ ]:
res_nsf = []
for col in L:
    nsf = pd.read_csv(f'range_benchmark/factors_nsf_prostate_poi_{col}_bench.csv',index_col=0)
    nsf = nsf.dropna(axis=1)
    nsf_aligned = nsf.reindex(adata.obs.index)
    adata.obsm['nsf'] = nsf_aligned
    print(col)
    find_leiden_resolution(adata, target_clusters=6,neighbors_key='nsf',key_added='NSF',use_rep='nsf')
    nsf_aligned['Histology'] = adata.obs['Histology']
    
    mut_info=metrics.fowlkes_mallows_score(adata.obs['NSF'], nsf_aligned ['Histology'])
    adj_rand=metrics.adjusted_rand_score(adata.obs['NSF'], nsf_aligned ['Histology'])
    adj_mut_info=metrics.adjusted_mutual_info_score(adata.obs['NSF'], nsf_aligned ['Histology'])
    res_nsf.append((col,mut_info,adj_rand,adj_mut_info))    

In [ ]:
score_nsf[2].max()

In [ ]:
score_nsf = pd.DataFrame(res_nsf)   
plt.plot(score_nsf[0],score_nsf[1],color="green",label='Fowlkes Mallows Score')
plt.plot(score_nsf[0],score_nsf[2],color="red",label='Adjusted Rand Index Score')
plt.plot(score_nsf[0],score_nsf[3],color="blue",label='Adjusted Mutual Info Score')
plt.xlabel("Number of Factors")
plt.ylabel("Score")
plt.title("NSF")
plt.legend(prop={'size': 9})

In [ ]:
res_nsfh = []
for col in L:
    nsfh_s = pd.read_csv(f'range_benchmark/spatialfactors_nsfh_prostate_poi_{col}_bench.csv',index_col=0)

    nsfh_ns = pd.read_csv(f'range_benchmark/nonspatialfactors_nsfh_prostate_poi_{col}_bench.csv',index_col=0)
    nsfh = pd.concat([nsfh_ns,nsfh_s], axis=1)
    nsfh = nsfh.dropna(axis=1)
    
    nsfh_aligned = nsfh.reindex(adata.obs.index)
    adata.obsm['nsfh'] = nsfh_aligned
    find_leiden_resolution(adata, target_clusters=6,neighbors_key='nsfh',key_added='NSFH',use_rep='nsfh')
    nsfh_aligned['Histology'] = adata.obs['Histology']
    
    mut_info=metrics.fowlkes_mallows_score(adata.obs['NSFH'], nsfh_aligned ['Histology'])
    adj_rand=metrics.adjusted_rand_score(adata.obs['NSFH'], nsfh_aligned ['Histology'])
    adj_mut_info=metrics.adjusted_mutual_info_score(adata.obs['NSFH'], nsfh_aligned ['Histology'])
    res_nsfh.append((col,mut_info,adj_rand,adj_mut_info)) 
    

In [ ]:
score_nsfh = pd.DataFrame(res_nsfh)   
plt.plot(score_nsfh[0],score_nsfh[1],color="green",label='Fowlkes Mallows Score')
plt.plot(score_nsfh[0],score_nsfh[2],color="red",label='Adjusted Rand Index Score')
plt.plot(score_nsfh[0],score_nsfh[3],color="blue",label='Adjusted Mutual Info Score')
plt.xlabel("Number of Factors")
plt.ylabel("Score")
plt.title("NSFH")
plt.legend(prop={'size': 9})

In [ ]:
score_nsfh[2].max()

In [ ]:
score_nsfh

In [ ]:
res_pnmf = []
for col in L:
    pnmf = pd.read_csv(f'range_benchmark/factors_pnmf_prostate_poi_{col}_bench.csv',index_col=0)
    pnmf = pnmf.dropna(axis=1)
    pnmf_aligned = pnmf.reindex(adata.obs.index)
    adata.obsm['pnmf'] = pnmf_aligned
    find_leiden_resolution(adata, target_clusters=6,neighbors_key='pnmf',key_added='PNMF',use_rep='pnmf')
    pnmf_aligned['Histology'] = adata.obs['Histology']
    
    mut_info=metrics.fowlkes_mallows_score(adata.obs['PNMF'],pnmf_aligned['Histology'])
    adj_rand=metrics.adjusted_rand_score(adata.obs['PNMF'],pnmf_aligned['Histology'])
    adj_mut_info=metrics.adjusted_mutual_info_score(adata.obs['PNMF'],pnmf_aligned['Histology'])
    res_pnmf.append((col,mut_info,adj_rand,adj_mut_info))  

In [ ]:
score_pnmf = pd.DataFrame(res_pnmf)   
plt.plot(score_pnmf[0],score_pnmf[1],color="green",label='Fowlkes Mallows Score')
plt.plot(score_pnmf[0],score_pnmf[2],color="red",label='Adjusted Rand Index Score')
plt.plot(score_pnmf[0],score_pnmf[3],color="blue",label='Adjusted Mutual Info Score')
plt.xlabel("Number of Factors")
plt.ylabel("Score")
plt.title("PNMF")
plt.legend(prop={'size': 9})
plt.savefig('/Users/user/Desktop/CellPie_paper/Revision/Revision_2/ARI_no_factors_pnmf_prostate.png')

In [ ]:
score_pnmf[2].max()

In [ ]:
score_pnmf

In [ ]:
L = np.arange(8,39)
L

In [ ]:
res_mefisto = []
for col in L:
    mef = pd.read_csv(f'range_benchmark/new_factors_mefisto_prostate_{col}_bench.csv',index_col=0)
    mef = mef.dropna(axis=1)
    mef_aligned = mef.reindex(adata.obs.index)
    adata.obsm['mefisto'] = mef_aligned
    find_leiden_resolution(adata, target_clusters=6,neighbors_key='mefisto',key_added='MEFISTO',use_rep='mefisto')
    mef_aligned['Histology'] = adata.obs['Histology']
    
    mut_info=metrics.fowlkes_mallows_score(adata.obs['MEFISTO'],mef_aligned['Histology'])
    adj_rand=metrics.adjusted_rand_score(adata.obs['MEFISTO'],mef_aligned['Histology'])
    adj_mut_info=metrics.adjusted_mutual_info_score(adata.obs['MEFISTO'],mef_aligned['Histology'])
    res_mefisto.append((col,mut_info,adj_rand,adj_mut_info)) 

In [ ]:
score_mef[2].max()

In [ ]:
score_mef

In [ ]:
score_mef = pd.DataFrame(res_mefisto)   
plt.plot(score_mef[0],score_mef[1],color="green",label='Fowlkes Mallows Score')
plt.plot(score_mef[0],score_mef[2],color="red",label='Adjusted Rand Index Score')
plt.plot(score_mef[0],score_mef[3],color="blue",label='Adjusted Mutual Info Score')
plt.xlabel("Number of Factors")
plt.ylabel("Score")
plt.title("MEFISTO")
plt.legend(prop={'size': 9})

In [ ]:
score_mef

In [ ]:
L = np.arange(8,51)
L

In [ ]:
from sklearn.decomposition import FactorAnalysis
res_fa = []
for col in L:
    transformer = FactorAnalysis(n_components=col, random_state=random_state)
    X_transformed = transformer.fit_transform(adata.X.toarray())
    adata.obsm['fa'] = X_transformed
    find_leiden_resolution(adata, target_clusters=6,neighbors_key='fa',key_added='FA',use_rep='fa')
    
    mut_info=metrics.fowlkes_mallows_score(adata.obs['FA'],adata.obs['Histology'])
    adj_rand=metrics.adjusted_rand_score(adata.obs['FA'],adata.obs['Histology'])
    adj_mut_info=metrics.adjusted_mutual_info_score(adata.obs['FA'],adata.obs['Histology'])
    res_fa.append((col,mut_info,adj_rand,adj_mut_info)) 

In [ ]:
score_fa = pd.DataFrame(res_fa)   
plt.plot(score_fa[0],score_fa[1],color="green",label='Fowlkes Mallows Score')
plt.plot(score_fa[0],score_fa[2],color="red",label='Adjusted Rand Index Score')
plt.plot(score_fa[0],score_fa[3],color="blue",label='Adjusted Mutual Info Score')
plt.xlabel("Number of Factors")
plt.ylabel("Score")
plt.title("FA")
plt.legend(prop={'size': 9})

In [ ]:
score_fa

In [ ]:
score_fa[2].max()

In [ ]:
# score_fa.to_csv('fa_range_of_factors_leiden_results_paper.csv')
# score_nsf.to_csv('nsf_range_of_factors_leiden_results_paper.csv')
# score_nsfh.to_csv('nsfh_range_of_factors_leiden_results_paper.csv')
# score_mef.to_csv('mefisto_range_of_factors_leiden_results_paper.csv')